# 📗 Phase 2 – Representation: 5️⃣ Normalization

## Giảng viên: Deep Learning với PyTorch

---

## Mục tiêu học tập

Sau khi hoàn thành notebook này, bạn sẽ:
- ✅ Hiểu **tại sao** normalization hoạt động (Internal Covariate Shift, gradient flow)
- ✅ Nắm vững **Batch Normalization**: train vs inference, running statistics
- ✅ Hiểu **Layer Normalization** và tại sao nó tốt cho Transformers
- ✅ Khám phá **RMSNorm**: hiệu quả và trade-offs
- ✅ Thực hành so sánh BN/LN/RMSNorm qua experiments

---

In [ ]:
# Import các thư viện cần thiết
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set style cho đẹp
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Seed để reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Kiểm tra GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Device: {device}")
print(f"🔥 PyTorch version: {torch.__version__}")

---

## 5.1 Tại sao Normalization hoạt động?

### 🎯 Internal Covariate Shift (Quan điểm lịch sử)

**Định nghĩa**: Sự thay đổi phân phối của input trong mỗi layer khi training.

**Vấn đề**:
- Layer sau phải "học lại" khi phân phối input thay đổi
- Làm chậm quá trình training
- Cần learning rate nhỏ để ổn định

### 🎯 Smoothing Optimization Landscape

**Nghiên cứu gần đây** (Santurkar et al., 2018):
- BatchNorm làm mượt loss landscape
- Gradient trở nên predictable hơn
- Cho phép learning rate lớn hơn

### 🎯 Stabilizing Gradient Flow

- Ngăn gradient vanishing/exploding
- Giữ gradient trong khoảng hợp lý
- Training sâu hơn, nhanh hơn

In [ ]:
# Visualization: Vẽ phân phối activation qua các layer
def visualize_activation_distribution(model, x, title="Activation Distribution"):
    """
    Visualize how activation distributions change across layers
    """
    activations = []
    
    # Forward pass và lưu activations
    with torch.no_grad():
        a = x
        for layer in model:
            a = layer(a)
            activations.append(a.cpu().numpy().flatten())
    
    # Vẽ histogram
    fig, axes = plt.subplots(1, len(activations), figsize=(15, 3))
    fig.suptitle(title, fontsize=16, fontweight='bold')
    
    for idx, (ax, act) in enumerate(zip(axes, activations)):
        ax.hist(act, bins=50, alpha=0.7, color=f'C{idx}')
        ax.set_title(f'Layer {idx+1}\nμ={act.mean():.3f}\nσ={act.std():.3f}')
        ax.set_xlabel('Activation value')
        ax.set_ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()

# Demo: So sánh network với và không có normalization
print("🔬 Demo: Internal Covariate Shift\n")

# Tạo sample data
x_sample = torch.randn(1000, 100)

# Network KHÔNG có normalization
model_no_norm = nn.Sequential(
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU()
)

# Network CÓ normalization
model_with_norm = nn.Sequential(
    nn.Linear(100, 100),
    nn.BatchNorm1d(100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.BatchNorm1d(100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.BatchNorm1d(100),
    nn.ReLU()
)

# Visualize
visualize_activation_distribution(
    [model_no_norm[0], model_no_norm[2], model_no_norm[4]], 
    x_sample,
    "❌ KHÔNG có Normalization - Phân phối bị shift"
)

visualize_activation_distribution(
    [model_with_norm[0:3], model_with_norm[3:6], model_with_norm[6:9]], 
    x_sample,
    "✅ CÓ Normalization - Phân phối ổn định"
)

### 📊 Nhận xét:

- **Không có Norm**: Activations bị shift, variance thay đổi qua layers
- **Có Norm**: Activations ổn định, mean≈0, std≈1 ở mỗi layer

---

## 5.2 Batch Normalization (BN)

### 📐 Công thức toán học

**Training mode**:
$$
\begin{align}
\mu_B &= \frac{1}{m}\sum_{i=1}^{m} x_i \quad \text{(mean của batch)} \\
\sigma_B^2 &= \frac{1}{m}\sum_{i=1}^{m} (x_i - \mu_B)^2 \quad \text{(variance của batch)} \\
\hat{x}_i &= \frac{x_i - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}} \quad \text{(normalize)} \\
y_i &= \gamma \hat{x}_i + \beta \quad \text{(scale & shift - learnable)}
\end{align}
$$

**Inference mode**:
$$
\hat{x} = \frac{x - \mu_{running}}{\sqrt{\sigma_{running}^2 + \epsilon}}
$$

### 🔄 Train vs Inference

| Mode | Mean & Variance | Cập nhật |
|------|----------------|----------|
| **Train** | Tính từ batch hiện tại | Cập nhật running statistics |
| **Inference** | Dùng running statistics | Không cập nhật |

In [ ]:
# Demo: BatchNorm từ scratch để hiểu rõ mechanism
class BatchNorm1dFromScratch(nn.Module):
    """
    Batch Normalization implementation from scratch để hiểu rõ cơ chế
    """
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        super().__init__()
        self.eps = eps
        self.momentum = momentum
        
        # Learnable parameters
        self.gamma = nn.Parameter(torch.ones(num_features))
        self.beta = nn.Parameter(torch.zeros(num_features))
        
        # Running statistics (không học, chỉ track)
        self.register_buffer('running_mean', torch.zeros(num_features))
        self.register_buffer('running_var', torch.ones(num_features))
        self.register_buffer('num_batches_tracked', torch.tensor(0, dtype=torch.long))
    
    def forward(self, x):
        # x shape: (batch_size, num_features)
        
        if self.training:
            # TRAINING MODE: tính mean & var từ batch
            batch_mean = x.mean(dim=0)
            batch_var = x.var(dim=0, unbiased=False)
            
            # Normalize
            x_normalized = (x - batch_mean) / torch.sqrt(batch_var + self.eps)
            
            # Cập nhật running statistics
            with torch.no_grad():
                self.running_mean = (1 - self.momentum) * self.running_mean + \
                                   self.momentum * batch_mean
                self.running_var = (1 - self.momentum) * self.running_var + \
                                  self.momentum * batch_var
                self.num_batches_tracked += 1
        else:
            # INFERENCE MODE: dùng running statistics
            x_normalized = (x - self.running_mean) / torch.sqrt(self.running_var + self.eps)
        
        # Scale & shift
        out = self.gamma * x_normalized + self.beta
        return out

print("🧪 Demo: BatchNorm Train vs Inference\n")

# Tạo data
x_train = torch.randn(32, 10) * 2 + 5  # mean≈5, std≈2

# Initialize BN layer
bn_custom = BatchNorm1dFromScratch(10)
bn_pytorch = nn.BatchNorm1d(10)

# Training mode
bn_custom.train()
bn_pytorch.train()
out_custom_train = bn_custom(x_train)
out_pytorch_train = bn_pytorch(x_train)

print("📊 TRAINING MODE:")
print(f"Input - Mean: {x_train.mean():.3f}, Std: {x_train.std():.3f}")
print(f"Output (Custom) - Mean: {out_custom_train.mean():.3f}, Std: {out_custom_train.std():.3f}")
print(f"Output (PyTorch) - Mean: {out_pytorch_train.mean():.3f}, Std: {out_pytorch_train.std():.3f}")
print(f"Running Mean (Custom): {bn_custom.running_mean.mean():.3f}")

# Inference mode với data khác
x_test = torch.randn(16, 10) * 3 + 10  # mean≈10, std≈3 (khác hẳn!)
bn_custom.eval()
bn_pytorch.eval()
out_custom_test = bn_custom(x_test)
out_pytorch_test = bn_pytorch(x_test)

print("\n📊 INFERENCE MODE:")
print(f"Input - Mean: {x_test.mean():.3f}, Std: {x_test.std():.3f}")
print(f"Output (Custom) - Mean: {out_custom_test.mean():.3f}, Std: {out_custom_test.std():.3f}")
print(f"Output (PyTorch) - Mean: {out_pytorch_test.mean():.3f}, Std: {out_pytorch_test.std():.3f}")
print(f"\n✅ Inference dùng running stats → output không bị ảnh hưởng bởi test batch!")

### 🔍 Batch Size Dependency

BatchNorm **phụ thuộc vào batch size**:
- Batch nhỏ → statistics không đáng tin
- Batch size khác nhau train/test → vấn đề!
- Thường cần batch size ≥ 16-32

In [ ]:
# Experiment: Ảnh hưởng của batch size lên BN
def test_batch_size_effect(batch_sizes=[2, 8, 32, 128]):
    """
    Test hiệu ứng của batch size lên BatchNorm
    """
    results = {}
    x_full = torch.randn(1000, 50)
    
    for bs in batch_sizes:
        bn = nn.BatchNorm1d(50)
        bn.train()
        
        # Process theo batch
        outputs = []
        for i in range(0, len(x_full), bs):
            batch = x_full[i:i+bs]
            if len(batch) == bs:  # Chỉ lấy batch đủ size
                out = bn(batch)
                outputs.append(out)
        
        all_outputs = torch.cat(outputs)
        results[bs] = {
            'mean': all_outputs.mean().item(),
            'std': all_outputs.std().item()
        }
    
    return results

print("🔬 Experiment: Ảnh hưởng của Batch Size\n")
results = test_batch_size_effect()

for bs, stats in results.items():
    print(f"Batch Size {bs:3d}: Mean={stats['mean']:6.3f}, Std={stats['std']:.3f}")

print("\n📌 Nhận xét: Batch size nhỏ → statistics không ổn định!")

### 🖼️ BatchNorm trong CNN vs MLP

**CNN**: `BatchNorm2d`
- Normalize theo (N, H, W) - giữ nguyên spatial structure
- Input shape: `(N, C, H, W)`
- Statistics shape: `(C,)` - mỗi channel có riêng mean/var

**MLP**: `BatchNorm1d`
- Normalize theo batch dimension
- Input shape: `(N, features)`
- Statistics shape: `(features,)`

In [ ]:
# Demo: BatchNorm2d cho CNN
print("🖼️  Demo: BatchNorm2d trong CNN\n")

# Tạo fake image data
images = torch.randn(8, 3, 32, 32)  # (batch, channels, height, width)
print(f"Input shape: {images.shape}")

# BatchNorm2d
bn2d = nn.BatchNorm2d(3)  # 3 channels
bn2d.train()
output = bn2d(images)

print(f"Output shape: {output.shape}")
print(f"\nGamma shape (learnable): {bn2d.weight.shape}")
print(f"Beta shape (learnable): {bn2d.bias.shape}")
print(f"Running mean shape: {bn2d.running_mean.shape}")
print(f"Running var shape: {bn2d.running_var.shape}")

# Verify normalization per channel
print("\n📊 Statistics per channel:")
for c in range(3):
    channel_data = output[:, c, :, :]
    print(f"Channel {c}: Mean={channel_data.mean():.3f}, Std={channel_data.std():.3f}")

---

## 5.3 Layer Normalization (LN)

### 📐 Công thức

$$
\begin{align}
\mu &= \frac{1}{D}\sum_{i=1}^{D} x_i \quad \text{(mean per sample)} \\
\sigma^2 &= \frac{1}{D}\sum_{i=1}^{D} (x_i - \mu)^2 \quad \text{(variance per sample)} \\
\hat{x}_i &= \frac{x_i - \mu}{\sqrt{\sigma^2 + \epsilon}} \\
y_i &= \gamma \hat{x}_i + \beta
\end{align}
$$

### 🔑 Khác biệt chính với BatchNorm:

| Aspect | BatchNorm | LayerNorm |
|--------|-----------|----------|
| Normalize theo | **Batch** dimension | **Feature** dimension |
| Phụ thuộc batch size | ✅ Có | ❌ Không |
| Train = Inference | ❌ Khác | ✅ Giống |
| Running statistics | ✅ Có | ❌ Không |
| Tốt cho | CNN, MLP | **Transformers, RNN** |

In [ ]:
# Visualization: BatchNorm vs LayerNorm
def visualize_norm_difference():
    """
    Visualize sự khác biệt giữa BatchNorm và LayerNorm
    """
    # Tạo data: (batch_size=4, features=6)
    x = torch.tensor([
        [1., 2., 3., 4., 5., 6.],
        [2., 4., 6., 8., 10., 12.],
        [3., 6., 9., 12., 15., 18.],
        [4., 8., 12., 16., 20., 24.]
    ])
    
    # Apply normalization
    bn = nn.BatchNorm1d(6)
    ln = nn.LayerNorm(6)
    
    bn.eval()
    ln.eval()
    
    x_bn = bn(x)
    x_ln = ln(x)
    
    # Plotting
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Original
    im0 = axes[0].imshow(x.numpy(), cmap='viridis', aspect='auto')
    axes[0].set_title('Original Data', fontweight='bold')
    axes[0].set_xlabel('Features')
    axes[0].set_ylabel('Samples')
    plt.colorbar(im0, ax=axes[0])
    
    # BatchNorm
    im1 = axes[1].imshow(x_bn.detach().numpy(), cmap='viridis', aspect='auto')
    axes[1].set_title('BatchNorm\n(normalize theo cột)', fontweight='bold')
    axes[1].set_xlabel('Features')
    axes[1].set_ylabel('Samples')
    plt.colorbar(im1, ax=axes[1])
    
    # LayerNorm
    im2 = axes[2].imshow(x_ln.detach().numpy(), cmap='viridis', aspect='auto')
    axes[2].set_title('LayerNorm\n(normalize theo hàng)', fontweight='bold')
    axes[2].set_xlabel('Features')
    axes[2].set_ylabel('Samples')
    plt.colorbar(im2, ax=axes[2])
    
    plt.tight_layout()
    plt.show()
    
    # Statistics
    print("📊 Statistics Verification:\n")
    print("BatchNorm - Mean per feature (should be ≈0):")
    print(x_bn.mean(dim=0).numpy().round(3))
    print("\nLayerNorm - Mean per sample (should be ≈0):")
    print(x_ln.mean(dim=1).numpy().round(3))

print("🎨 Visualization: BatchNorm vs LayerNorm\n")
visualize_norm_difference()

### 🤖 Tại sao LayerNorm tốt cho Transformers?

1. **Sequence length khác nhau**: Mỗi sample có thể có length khác
2. **Batch size nhỏ**: Transformers thường train với batch nhỏ (do memory)
3. **Train = Inference**: Không cần running statistics
4. **Per-token normalization**: Mỗi token được normalize độc lập

**Trong Transformer**:
```python
# Input shape: (batch, seq_len, d_model)
# LayerNorm normalize theo d_model dimension
ln = nn.LayerNorm(d_model)
```

In [ ]:
# Demo: LayerNorm trong Transformer-style architecture
class SimpleTransformerBlock(nn.Module):
    """
    Simplified Transformer block với LayerNorm
    """
    def __init__(self, d_model=512, nhead=8):
        super().__init__()
        self.attention = nn.MultiheadAttention(d_model, nhead, batch_first=True)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_model * 4),
            nn.ReLU(),
            nn.Linear(d_model * 4, d_model)
        )
    
    def forward(self, x):
        # Self-attention + residual + LayerNorm
        attn_out, _ = self.attention(x, x, x)
        x = self.norm1(x + attn_out)
        
        # FFN + residual + LayerNorm
        ffn_out = self.ffn(x)
        x = self.norm2(x + ffn_out)
        return x

print("🤖 Demo: LayerNorm trong Transformer\n")

# Tạo fake sequence data với varying lengths
batch_size = 4
seq_lengths = [10, 15, 8, 12]  # Khác nhau!
d_model = 512

# Pad sequences
max_len = max(seq_lengths)
x = torch.randn(batch_size, max_len, d_model)

# Create model
model = SimpleTransformerBlock(d_model=d_model)
model.eval()

# Forward pass
with torch.no_grad():
    output = model(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print(f"\n✅ LayerNorm hoạt động tốt với sequence lengths khác nhau!")
print(f"✅ Không cần batch statistics → inference ổn định!")

---

## 5.4 RMSNorm (Root Mean Square Normalization)

### 📐 Công thức

$$
\begin{align}
\text{RMS} &= \sqrt{\frac{1}{D}\sum_{i=1}^{D} x_i^2} \\
\hat{x}_i &= \frac{x_i}{\text{RMS} + \epsilon} \cdot \gamma
\end{align}
$$

### 🔑 Khác biệt với LayerNorm:

| Aspect | LayerNorm | RMSNorm |
|--------|-----------|----------|
| Mean centering | ✅ Có (`x - μ`) | ❌ **KHÔNG** |
| Variance normalization | ✅ Có | ✅ Có (via RMS) |
| Learnable bias β | ✅ Có | ❌ Không |
| Computation | 2 passes | **1 pass** |
| Speed | Slower | **~15% faster** |

### 💡 Tại sao bỏ mean?

- **Empirical finding**: Mean centering không quan trọng lắm!
- **Efficiency**: Giảm computation
- **Được dùng trong**: LLaMA, GPT-NeoX, T5

In [ ]:
# Implementation: RMSNorm from scratch
class RMSNorm(nn.Module):
    """
    Root Mean Square Layer Normalization
    Paper: https://arxiv.org/abs/1910.07467
    """
    def __init__(self, dim, eps=1e-6):
        super().__init__()
        self.eps = eps
        self.gamma = nn.Parameter(torch.ones(dim))
    
    def forward(self, x):
        # Compute RMS
        rms = torch.sqrt(torch.mean(x ** 2, dim=-1, keepdim=True) + self.eps)
        
        # Normalize and scale
        x_normalized = x / rms
        return self.gamma * x_normalized

print("🔬 Implementation: RMSNorm\n")

# Test data
x_test = torch.randn(4, 8, 512)  # (batch, seq, features)

# Initialize
ln = nn.LayerNorm(512)
rms = RMSNorm(512)

# Forward pass
with torch.no_grad():
    out_ln = ln(x_test)
    out_rms = rms(x_test)

print(f"Input shape: {x_test.shape}")
print(f"\nLayerNorm output:")
print(f"  Mean: {out_ln.mean():.6f} (should be ≈0)")
print(f"  Std: {out_ln.std():.6f}")
print(f"\nRMSNorm output:")
print(f"  Mean: {out_rms.mean():.6f} (NOT zero!)")
print(f"  Std: {out_rms.std():.6f}")
print(f"\n📌 RMSNorm KHÔNG center về 0 → faster computation!")

### ⚖️ Trade-offs

**Advantages**:
- ⚡ Faster (~15% speedup)
- 💾 Less memory (no mean tracking)
- 🎯 Simpler gradient computation

**Potential Issues**:
- ⚠️ Không center về 0 → có thể ảnh hưởng một số architectures
- ⚠️ Ít được research hơn LayerNorm
- ⚠️ Phải tune hyperparameters lại khi switch từ LN

---

## 5.5 Practical Experiments: So sánh BN / LN / RMSNorm

Chúng ta sẽ train cùng 1 model với 3 loại normalization khác nhau và so sánh:
- ⚡ Convergence speed
- 📊 Batch size sensitivity
- 📈 Gradient norm stability

In [ ]:
# Tạo synthetic dataset
def create_classification_dataset(n_samples=10000, n_features=50, n_classes=10):
    """
    Tạo synthetic classification dataset
    """
    X = torch.randn(n_samples, n_features)
    # Tạo labels với some structure
    W = torch.randn(n_features, n_classes)
    logits = X @ W
    y = logits.argmax(dim=1)
    
    return TensorDataset(X, y)

# Define models với các normalization khác nhau
class MLPWithNorm(nn.Module):
    """
    Simple MLP với pluggable normalization
    """
    def __init__(self, input_dim=50, hidden_dim=128, output_dim=10, norm_type='bn'):
        super().__init__()
        self.norm_type = norm_type
        
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        
        # Normalization layers
        if norm_type == 'bn':
            self.norm1 = nn.BatchNorm1d(hidden_dim)
            self.norm2 = nn.BatchNorm1d(hidden_dim)
        elif norm_type == 'ln':
            self.norm1 = nn.LayerNorm(hidden_dim)
            self.norm2 = nn.LayerNorm(hidden_dim)
        elif norm_type == 'rms':
            self.norm1 = RMSNorm(hidden_dim)
            self.norm2 = RMSNorm(hidden_dim)
        else:
            self.norm1 = nn.Identity()
            self.norm2 = nn.Identity()
    
    def forward(self, x):
        x = F.relu(self.norm1(self.fc1(x)))
        x = F.relu(self.norm2(self.fc2(x)))
        x = self.fc3(x)
        return x

print("🏗️  Setup: Creating models and dataset...\n")
dataset = create_classification_dataset()
print(f"✅ Dataset created: {len(dataset)} samples")

In [ ]:
# Training function với gradient tracking
def train_model(model, dataloader, epochs=10, lr=0.001, device='cpu'):
    """
    Train model và track metrics
    """
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    history = {
        'loss': [],
        'accuracy': [],
        'grad_norm': []
    }
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0
        grad_norms = []
        
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            
            optimizer.zero_grad()
            outputs = model(X)
            loss = criterion(outputs, y)
            loss.backward()
            
            # Track gradient norm
            total_norm = 0
            for p in model.parameters():
                if p.grad is not None:
                    total_norm += p.grad.data.norm(2).item() ** 2
            grad_norms.append(total_norm ** 0.5)
            
            optimizer.step()
            
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(y).sum().item()
            total += y.size(0)
        
        avg_loss = total_loss / len(dataloader)
        accuracy = 100. * correct / total
        avg_grad_norm = np.mean(grad_norms)
        
        history['loss'].append(avg_loss)
        history['accuracy'].append(accuracy)
        history['grad_norm'].append(avg_grad_norm)
    
    return history

print("🎯 Training function ready!")

In [ ]:
# Experiment 1: Convergence speed với batch size = 32
print("🚀 Experiment 1: Convergence Speed\n")

batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Train các models
results = {}
norm_types = ['none', 'bn', 'ln', 'rms']
colors = ['gray', 'blue', 'green', 'red']

for norm_type in tqdm(norm_types, desc="Training models"):
    model = MLPWithNorm(norm_type=norm_type)
    history = train_model(model, dataloader, epochs=20, device=device)
    results[norm_type] = history

# Plotting
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('📊 Convergence Comparison (Batch Size = 32)', fontsize=16, fontweight='bold')

# Loss
for norm_type, color in zip(norm_types, colors):
    axes[0].plot(results[norm_type]['loss'], label=norm_type.upper(), 
                color=color, linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy
for norm_type, color in zip(norm_types, colors):
    axes[1].plot(results[norm_type]['accuracy'], label=norm_type.upper(),
                color=color, linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_title('Training Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Gradient norm
for norm_type, color in zip(norm_types, colors):
    axes[2].plot(results[norm_type]['grad_norm'], label=norm_type.upper(),
                color=color, linewidth=2)
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Gradient Norm')
axes[2].set_title('Gradient Stability')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Observations:")
print("   - NONE: Slower convergence, unstable gradients")
print("   - BN/LN/RMS: Faster convergence, stable gradients")
print("   - RMS: Slightly faster than LN")

In [ ]:
# Experiment 2: Batch size sensitivity
print("🔬 Experiment 2: Batch Size Sensitivity\n")

batch_sizes = [8, 16, 32, 64]
batch_results = {bs: {} for bs in batch_sizes}

for bs in tqdm(batch_sizes, desc="Testing batch sizes"):
    dataloader = DataLoader(dataset, batch_size=bs, shuffle=True)
    
    for norm_type in ['bn', 'ln', 'rms']:
        model = MLPWithNorm(norm_type=norm_type)
        history = train_model(model, dataloader, epochs=10, device=device)
        batch_results[bs][norm_type] = history['accuracy'][-1]  # Final accuracy

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
fig.suptitle('📊 Batch Size Sensitivity', fontsize=16, fontweight='bold')

for norm_type, color in zip(['bn', 'ln', 'rms'], ['blue', 'green', 'red']):
    accuracies = [batch_results[bs][norm_type] for bs in batch_sizes]
    ax.plot(batch_sizes, accuracies, marker='o', label=norm_type.upper(),
           color=color, linewidth=2, markersize=8)

ax.set_xlabel('Batch Size', fontsize=12)
ax.set_ylabel('Final Accuracy (%)', fontsize=12)
ax.set_title('Effect of Batch Size on Performance')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xscale('log', base=2)

plt.tight_layout()
plt.show()

print("\n✅ Observations:")
print("   - BN: Sensitive to batch size (worse with small batches)")
print("   - LN/RMS: More robust across batch sizes")
print("   - RMS: Consistently good performance")

---

## 📚 Tổng kết: Khi nào dùng gì?

| Use Case | Recommended | Lý do |
|----------|-------------|-------|
| **CNN (Computer Vision)** | BatchNorm2d | Spatial structure, batch statistics reliable |
| **Transformers / NLP** | LayerNorm | Variable length, small batches |
| **RNN / LSTM** | LayerNorm | Temporal data, sequence-to-sequence |
| **Large Language Models** | RMSNorm | Efficiency, proven in LLaMA/GPT |
| **Small batch training** | LayerNorm / RMSNorm | Không phụ thuộc batch |
| **Inference optimization** | RMSNorm | Fastest, simplest |

### 💡 Best Practices:

1. **BatchNorm**:
   - Batch size ≥ 16-32
   - `.train()` và `.eval()` quan trọng!
   - Cẩn thận với distributed training

2. **LayerNorm**:
   - Safe default cho most cases
   - Đặc biệt tốt cho Transformers
   - Không cần worry về batch size

3. **RMSNorm**:
   - Thử khi cần optimize speed
   - Tune hyperparameters lại
   - Monitor training stability

---

## 🎓 Bài tập thực hành

1. **Implement GroupNorm** từ scratch (normalize theo groups of channels)
2. **Compare BN vs SyncBN** trong distributed training
3. **Implement InstanceNorm** cho style transfer
4. **Test normalization** với different activation functions

---

## 📖 References

1. [Batch Normalization Paper](https://arxiv.org/abs/1502.03167)
2. [Layer Normalization Paper](https://arxiv.org/abs/1607.06450)
3. [RMSNorm Paper](https://arxiv.org/abs/1910.07467)
4. [How Does Batch Normalization Help?](https://arxiv.org/abs/1805.11604)

---

### 🙏 Cảm ơn bạn đã học!

Next: **6️⃣ Activation Functions** 🔥